In [1]:
from tensorflow import keras as k

In [2]:
model = k.models.load_model('model_veoneer.h5')

In [3]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 114, 114, 32)      896       
                                                                 
 batch_normalization_21 (Bat  (None, 114, 114, 32)     128       
 chNormalization)                                                
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 57, 57, 32)       0         
 2D)                                                             
                                                                 
 batch_normalization_22 (Bat  (None, 57, 57, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_10 (Conv2D)          (None, 57, 57, 64)        18496     
                                                      

In [4]:
model.layers[0].input_shape

(None, 227, 227, 3)

In [5]:
model.layers[0].output_shape

(None, 114, 114, 32)

In [6]:
model.layers[0].get_weights()

[array([[[[ 2.57298257e-02,  8.20839852e-02,  7.42490441e-02,
            3.14451777e-03,  1.11567467e-01, -6.16517626e-02,
            6.72354102e-02, -9.24067125e-02, -1.04800969e-01,
            1.19255327e-01, -1.12456456e-02, -9.11614895e-02,
           -6.74000308e-02, -1.34174787e-02,  6.49204701e-02,
            6.01352379e-02,  1.51573708e-02,  3.36761810e-02,
           -9.91299897e-02,  4.95262258e-02,  1.24592736e-01,
            1.77753761e-01, -1.69892535e-01,  4.77935597e-02,
            1.22370206e-01, -1.07480183e-01,  1.76738407e-02,
           -1.13590568e-01,  7.12717846e-02, -7.37662241e-02,
            3.26874740e-02, -1.40494714e-02],
          [-1.14451721e-01, -4.28487733e-02, -2.61037815e-02,
           -1.18832469e-01, -6.50056973e-02, -1.11386791e-01,
           -3.07302400e-02, -2.63700765e-02, -1.00210682e-01,
            1.20404169e-01, -7.66967088e-02,  2.57455520e-02,
           -1.00968622e-01,  2.12322362e-02, -6.19779117e-02,
            5.92312813e-

In [8]:
import sounddevice as sd
import math

In [9]:
def convert_to_probabilities(predicted_values):
    exp_values = [math.exp(val) for val in predicted_values]
    sum_exp_values = sum(exp_values)
    probabilities = [val / sum_exp_values for val in exp_values]
    return probabilities

In [ ]:
fs = 44100
duration = 2 # seconds

val = sd.rec(frames=fs*duration, samplerate=fs, channels=1)
sd.wait()

